<a href="https://colab.research.google.com/github/FatimaJahara/FineGrainedBNArc/blob/main/news_stat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%time
!pip install ktrain
import ktrain
from ktrain import text

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from nltk import word_tokenize
import nltk

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import average_precision_score, roc_auc_score, roc_curve, precision_recall_curve

np.random.seed(0)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.3/25.3 MB 24.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 23.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 653.6/653.6 kB 29.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.8/468.8 kB 18.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Prepari

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
acc = pd.read_csv('/content/drive/My Drive/Data/Accident_journal.csv')
crime = pd.read_csv('/content/drive/My Drive/Data/Crime_journal.csv')
ent = pd.read_csv('/content/drive/My Drive/Data/Entertainment_journal.csv')
sports = pd.read_csv('/content/drive/My Drive/Data/Sports_journal.csv')

In [ ]:
acc

,ID,Title,Description,Label
0,1,পাখির আঘাতে ড্যাশ উড়োজাহাজের জরুরি অবতরণ,বিমান বাংলাদেশ এয়ারলাইনসের ড্যাশ মডেলের উড়োজাহ...,Air
1,2,চট্টগ্রামে বিমানে যান্ত্রিক ত্রুটি জরুরি অবতরণ,আকাশে ওড়ার মিনিট চট্টগ্রাম শাহ আমানত আন্তর্জ...,Air
2,3,বিমানের ড্যাশ এইটের জরুরি অবতরণ,হজরত শাহজালাল আন্তর্জাতিক বিমানবন্দরে বিমান বা...,Air
3,4,উড়োজাহাজেই মৃত্যু ওমানপ্রবাসীর ঘটেছি ফ্লাইটে,স্বপ্ন বিদেশে গিয়েছিলেন ওমান প্রবাসী রানা আহমে...,Air
4,5,অভ্র করবেন,ইয়াঙ্গুন আন্তর্জাতিক বিমানবন্দরে মে সন্ধ্যায় ...,Air
...,...,...,...,...
11842,11843,সাঙ্গু নদে ডুবে কলেজছাত্রের মৃত্যু,বান্দরবানের মার্মা বাজার ঘাটে সাঙ্গু নদে ডুবে ...,Water
11843,11844,পানিতে ডুবে গৃহবধূর মৃত্যু,ফরিদপুরে পানিতে ডুবে মারা গেছেন গৃহবধূ ফরিদা ব...,Water
11844,11845,উদ্ধার হননি লঞ্চ থে‌কে প‌ড়ে নারী,ঢাকাবরিশালগামী লঞ্চ গতকাল শনিবার রাতে পা ফসকে ...,Water
11845,11846,উদ্ধার জেলে খোঁজ মেলেনি নৌকার,সেন্ট মার্টিন টেকনাফে ফেরার পথে পর্যটকবাহী জাহ...,Water


In [ ]:
acc['cat_label'] = 0
crime['cat_label'] = 1
ent['cat_label'] = 3
sports['cat_label'] = 4

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
def label_encoder(df):

    df["Label"] = le.fit_transform(df["Label"].astype(str))
    label_map = dict(zip(le.classes_, le.transform(le.classes_)))
    print(label_map)


    df["Label"] = le.fit_transform(df["Label"].astype(str))
    label_map = dict(zip(le.classes_, le.transform(le.classes_)))
    print(label_map)

label_encoder(acc)
label_encoder(crime)
label_encoder(ent)
label_encoder(sports)

{'Air': 0, 'Blast': 1, 'Construction': 2, 'Electricity': 3, 'Fire': 4, 'Rail': 5, 'Road': 6, 'Water': 7, 'others': 8}
{'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8}
{'CorruptionandFraud': 0, 'Drug': 1, 'Murder': 2, 'Others': 3, 'RapeandAbuse': 4, 'TheftRobberyAbduction': 5, 'Trafficking': 6}
{'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6}
{'bollywood': 0, 'dhallywood': 1, 'hollywood': 2, 'music': 3, 'television': 4, 'tollywood': 5}
{'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5}
{'athletics': 0, 'cricket': 1, 'football': 2, 'tennis': 3}
{'0': 0, '1': 1, '2': 2, '3': 3}


In [ ]:
accY = acc["Label"]
accX = acc['Description']

crimeY = crime["Label"]
crimeX = crime["Description"]

entY = ent["Label"]
entX = ent["Description"]

sportsY = sports["Label"]
sportsX = sports["Description"]

In [ ]:
accX_train, accX_test, accy_train, accy_test = train_test_split(accX, accY, test_size = 0.1, random_state = 5 , stratify=accY)
crimeX_train, crimeX_test, crimey_train, crimey_test = train_test_split(crimeX, crimeY, test_size = 0.1, random_state = 4,  stratify=crimeY)
entX_train, entX_test, enty_train, enty_test = train_test_split(entX, entY, test_size = 0.1, random_state = 5, stratify=entY)
sportsX_train, sportsX_test, sportsy_train, sportsy_test = train_test_split(sportsX, sportsY, test_size = 0.1, random_state = 5, stratify=sportsY)

In [ ]:
stopwords =list(open('/content/drive/My Drive/Data/Stopwords.txt', mode='r', encoding='UTF-8'))

nltk.download('punkt')

def remove_stopwords(text):
    removed = []
    stop_words = stopwords
    tokens = word_tokenize(text)
    for i in range(len(tokens)):
        if tokens[i] not in stop_words:
            removed.append(tokens[i])
    return " ".join(removed)

acc['Description'] = acc['Description'].apply(lambda x: remove_stopwords(str(x)))
acc['Title'] = acc['Title'].apply(lambda x: remove_stopwords(str(x)))
acc['New_length'] = acc['Description'].apply(lambda x: len(str(x)))
acc = acc.astype({'New_length': np.int})

crime['Description'] = crime['Description'].apply(lambda x: remove_stopwords(str(x)))
crime['Title'] = crime['Title'].apply(lambda x: remove_stopwords(str(x)))
crime['New_length'] = crime['Description'].apply(lambda x: len(str(x)))
crime = crime.astype({'New_length': np.int})

ent['Description'] = ent['Description'].apply(lambda x: remove_stopwords(str(x)))
ent['Title'] = ent['Title'].apply(lambda x: remove_stopwords(str(x)))
ent['New_length'] = ent['Description'].apply(lambda x: len(str(x)))
ent = ent.astype({'New_length': np.int})

sports['Description'] = sports['Description'].apply(lambda x: remove_stopwords(str(x)))
sports['Title'] = sports['Title'].apply(lambda x: remove_stopwords(str(x)))
sports['New_length'] = sports['Description'].apply(lambda x: len(str(x)))
sports = sports.astype({'New_length': np.int})